# Yajemati Wall

In [1]:
import urllib.request
import json
import pandas as pd
from token_key import k
import ssl
import re
import random
regex = re.compile("[А-ЯЁЙЁ]{3,}") 

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

## 0. Функция, читающая url

In [3]:
def readUrl(url, key=k):
    url = url + k
    content = urllib.request.urlopen(url, context=ctx)
    decoded_content = content.read().decode("utf-8")
    data = json.loads(decoded_content)
    return data

## 1. Собираем все id постов со страницы

In [4]:
def post_numbers(owner_id:str, offset_limit:int):
    data = readUrl("https://api.vk.com/method/wall.get?owner_id={}&count=100&v=5.95&access_token=".format(owner_id))
    post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
    offset = 101
    while offset < offset_limit:
        data = readUrl("https://api.vk.com/method/wall.get?owner_id={}&offset={}&count=100&v=5.95&access_token=".format(owner_id, offset))
        new_post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
        post_ids.update(new_post_ids)
        offset += 100
    return post_ids

In [5]:
post_ids = post_numbers(owner_id="-143177265", 
                        offset_limit=10002)

In [6]:
len(post_ids)

10100

## 2. Собираем все комментарии с капслоком на русском языке по пост-id

In [7]:
def get_comments(owner_id:str,  
                 post_ids:list,
                offset_limit=1002):

    result_entries = []
    for id_ in post_ids:
        offset = 1
        while offset < offset_limit: 
            data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&post_id={}&offset={}&count=100&v=6.7&access_token=".format(owner_id, str(id_), offset))
            try:
                data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                data_list = data.values.tolist()
                lines = [i[2] for i in data_list]
                offset += 100
                for i, line in enumerate(lines):
                    try:
                        if re.search(regex, line):
                            result_entries.append(data_list[i])
                    except TypeError:
                        continue
                
                comment_ids = [i[3] for i in data_list]
                for com_id in comment_ids:
                    comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&comment_id={}&count=100&v=6.7&access_token=".format(owner_id, com_id))
                    comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                    comment_data_list = comment_data.values.tolist()
                    lines2 = [i[2] for i in comment_data_list]
                    for i, line in enumerate(lines2):
                        try:
                            if re.search(regex, line):
                                result_entries.append(comment_data_list[i])
                        except TypeError:
                            pass
                        
                if offset%100 == 1:
                    print(offset, "ids parsed")

            except KeyError:
                continue
                
    return result_entries

In [18]:
#capslock_comments = get_comments(owner_id="-145800770",
#                                 post_ids=post_ids,
#                                 offset_limit=1002)

In [12]:
offset_limit = 1001
owner_id = "-143177265"

result_entries = []
count = 0
count_ids = 0
for id_, text in post_ids.items():
    offset = 1
    while offset < offset_limit: 
        data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&post_id={}&offset={}&count=100&v=6.7&access_token=".format(owner_id, id_, offset))
        try:
            data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
            data_list = data.values.tolist()
            for i, lst in enumerate(data_list):
                data_list[i].append(text)
            lines = [i[2] for i in data_list]
            offset += 100
            for i, line in enumerate(lines):
                try:
                    if re.search(regex, line):
                        result_entries.append(data_list[i])
                        count += 1
                        if count%100 == 0:
                            print(count, "comments with capslock collected")
                except TypeError:
                    continue
                
            comment_ids_and_text = {i[3]:i[2] for i in data_list}
            for com_id, com_text in comment_ids_and_text.items():
                comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&comment_id={}&count=100&v=6.7&access_token=".format(owner_id, com_id))
                comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "parents_stack", "text", "id"])
                comment_data_list = comment_data.values.tolist()
                for i, lst in enumerate(comment_data_list):
                    comment_data_list[i].append(com_text)
                lines2 = [i[2] for i in comment_data_list]
                for i, line in enumerate(lines2):
                    try:
                        if re.search(regex, line):
                            result_entries.append(comment_data_list[i])
                            count += 1
                            if count%100 == 0:
                                print(count, "comments with capslock collected")
                    except TypeError:
                        pass
        except KeyError:
            continue
    count_ids += 1
    if count_ids % 50 == 0:
        print(count_ids, "ids proceeded")

100 comments with capslock collected
50 ids proceeded
200 comments with capslock collected
300 comments with capslock collected
100 ids proceeded
400 comments with capslock collected
500 comments with capslock collected
600 comments with capslock collected
150 ids proceeded
700 comments with capslock collected
800 comments with capslock collected
200 ids proceeded
900 comments with capslock collected
1000 comments with capslock collected
250 ids proceeded
1100 comments with capslock collected
1200 comments with capslock collected
1300 comments with capslock collected
300 ids proceeded
1400 comments with capslock collected
1500 comments with capslock collected
350 ids proceeded
1600 comments with capslock collected
1700 comments with capslock collected
400 ids proceeded
1800 comments with capslock collected
1900 comments with capslock collected
2000 comments with capslock collected
2100 comments with capslock collected
450 ids proceeded
2200 comments with capslock collected
2300 comment

18600 comments with capslock collected
18700 comments with capslock collected
18800 comments with capslock collected
18900 comments with capslock collected
3000 ids proceeded
19000 comments with capslock collected
19100 comments with capslock collected
19200 comments with capslock collected


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [13]:
df_res = pd.DataFrame(result_entries, columns = ["from_id", "parent_id", "comment_text", "comment_id", "parent_text"])
df_res.count()

# 2550 ids proceeded
# 31700 comments with capslock collected
# 31800 comments with capslock collected

from_id         19276
parent_id       19276
comment_text    19276
comment_id      19276
parent_text     19037
dtype: int64

## 4. Извлекаем инфу о каждом комментаторе и добавляем в датафрейм. 

In [14]:
def chunks(list_of_values, chunk_size=400):
    return [list_of_values[i:i + chunk_size] for i in range(0, len(list_of_values), chunk_size)]

In [15]:
def update_df_with_user_info(df):
    
    df["first_name"] = None 
    df["last_name"] = None 
    df["sex"] = None
    df["bdate"] = None 
    df["city"] = None
    
    df.drop_duplicates(subset=["from_id"], keep='first', inplace=True)
    df["from_id"] = df["from_id"].apply(lambda x: int(x))
    from_ids = [str(df.from_id[i]) for i in df.index]
    df = df.set_index("from_id")
        
    all_data = []
    for ids in chunks(from_ids):
        k = ",".join(ids)
        data = readUrl("https://api.vk.com/method/users.get?user_ids={}&fields=bdate,sex,city&v=6.7&access_token=".format(str(k)))
        all_data += data["response"]
    
    print(all_data[0])
    
    for el in all_data:
        indx = el["id"]
        try:
            el["city"] = el["city"]["title"]
        except KeyError:
            continue
        df.loc[indx, "first_name"] = el.get("first_name", None)
        df.loc[indx, "last_name"] = el.get("last_name", None)
        df.loc[indx, "sex"] = el.get("sex", None)
        df.loc[indx,"bdate"] = el.get("bdate", None)
        df.loc[indx,"city"] = el.get("city", None)
        
    return df

In [16]:
df_res = update_df_with_user_info(df_res)

{'id': 435152181, 'first_name': 'Dima', 'last_name': 'Kirichevsky', 'is_closed': False, 'can_access_closed': True, 'sex': 2, 'bdate': '26.6', 'city': {'id': 10, 'title': 'Volgograd'}}


In [17]:
df_res.tail()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city
from_id,,,,,,,,,
330060237,6711556,"Дааа. "" У меня есть дырка между ног! Да, все с...",6711952,,Deniel,Semenov,2,None,Baley
105574664,6711556,"Как жаль, что раньше это была группа ПРО яжмат...",6712341,,Rita,Gusarova,1,8.6.1997,Saint Petersburg
351283909,6711556,"[id92358042|Яна], боже,я высказала СВОЕ мнение...",6712914,,None,None,None,None,None
493442372,6711556,В США везде можно попросить бесплатно воды. В ...,6714414,,None,None,None,None,None
353977671,6711556,"[id12178950|Леонид], не ну орать на фармацевта...",6715225,,None,None,None,None,None


## 5. Удалим комменты с пустыми полями sex, bdate, city

In [18]:
df_test = df_res.dropna(subset=["sex", "bdate", "city"],how="all") 

In [19]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6613 entries, 435152181 to 105574664
Data columns (total 9 columns):
parent_id       6613 non-null object
comment_text    6613 non-null object
comment_id      6613 non-null int64
parent_text     6535 non-null object
first_name      6613 non-null object
last_name       6613 non-null object
sex             6613 non-null object
bdate           5439 non-null object
city            6613 non-null object
dtypes: int64(1), object(8)
memory usage: 310.0+ KB


In [20]:
df_test.sex.value_counts()
# со 100 постов собралось всего 287 комментариев с капслоком 

1    3322
2    3289
0       2
Name: sex, dtype: int64

In [21]:
df_test.city.value_counts()

Moscow              1153
Saint Petersburg     581
Yekaterinburg        119
Novosibirsk          116
Minsk                107
Chelyabinsk           92
Krasnodar             91
Samara                86
Nizhny Novgorod       82
Voronezh              74
Rostov-on-Don         68
Volgograd             66
Krasnoyarsk           65
Omsk                  59
Kyiv                  58
Perm                  55
Kazan                 55
Khabarovsk            47
Ufa                   47
Tomsk                 46
Kaliningrad           46
Tyumen                45
Vladivostok           44
Ryazan                40
Kemerovo              39
Donetsk               38
Saratov               36
Irkutsk               35
Penza                 35
Almaty                32
                    ... 
Sursk                  1
Semenovskoe            1
Tbilisi                1
Lisboa                 1
Antananarivo           1
Klintsy                1
Białystok              1
Edinburgh              1
Dedovsk                1


In [22]:
df_test.head()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city
from_id,,,,,,,,,
435152181,6.93346e+06,Правильно нехер таких дебилов на воли держать-...,6933869,,Dima,Kirichevsky,2,26.6,Volgograd
512031769,[6933469],"[id461371840|Алдар], смотря каким образом спро...",7200972,Да,Irina,Storozhenko,1,21.1.2000,Rostov-on-Don
210549261,[6933469],"[id270545448|Софья], УАХАХАХАХАХ, если бы толь...",7340519,Да,Andrey,Polovinkin,2,20.1.2000,Nizhny Novgorod
161186753,[6933470],"[id270545448|Софья], я сказала, что она МОГЛА ...",6933570,Может девочка сама виновата?,Anastasia,Kim,1,None,Tobolsk
320620379,[6933470],"[id161186753|Анастасия], камон, ЛЮБОЕ поведени...",6933722,Может девочка сама виновата?,Varya,Tanvel,1,7.10.1917,Moscow


In [23]:
df_test = df_test.reset_index()

In [24]:
df_test = df_test.drop_duplicates(subset=["from_id"])
df_test.count()

from_id         6613
parent_id       6613
comment_text    6613
comment_id      6613
parent_text     6535
first_name      6613
last_name       6613
sex             6613
bdate           5439
city            6613
dtype: int64

## Пишем функцию, которая собирает инфу о группах юзера

In [25]:
def update_df_with_subskr_info(df, offset_limit=1001):
    
    df = df.set_index("from_id")
    df["group_info"] = None
    indices = list(df.index)
        
    count = 0
    
    for i in indices: 
        offset = 0
        group_info = []
        while offset < offset_limit:
            subskr = readUrl("https://api.vk.com/method/users.getSubscriptions?user_id={}&extended=1&fields=activity,status,description&count=200&offset={}&v=6.7&access_token=".format(str(i), offset))
            offset += 200
            try:
                for j in subskr["response"]["items"]:
                    group = [j["name"], j["status"], j["description"], j["activity"]]
                    group_info.append(group)
            except KeyError:
                continue
            
        if group_info != []:
            group_info = [" -||- ".join(i) for i in group_info]
            group_info = " -|-|- ".join(group_info)
            df.loc[i, "group_info"] = group_info
        
        count += 1
        if count % 100 == 0:
            print(count, "users processed")
        
    return df

In [26]:
df = update_df_with_subskr_info(df_test)

100 users processed
200 users processed
300 users processed
400 users processed
500 users processed
600 users processed
700 users processed
800 users processed
900 users processed
1000 users processed
1100 users processed
1200 users processed
1300 users processed
1400 users processed
1500 users processed
1600 users processed
1700 users processed
1800 users processed
1900 users processed
2000 users processed
2100 users processed
2200 users processed
2300 users processed
2400 users processed
2500 users processed
2600 users processed
2700 users processed
2800 users processed
2900 users processed
3000 users processed
3100 users processed
3200 users processed
3300 users processed
3400 users processed
3500 users processed
3600 users processed
3700 users processed
3800 users processed
3900 users processed
4000 users processed
4100 users processed
4200 users processed
4300 users processed
4400 users processed
4500 users processed
4600 users processed
4700 users processed
4800 users processed
4

In [27]:
df.tail()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info
from_id,,,,,,,,,,
259335134,6711754,"[id211787099|Антонина], именно. Но, к сожалени...",6712100,,Anastasia,Uryutina,1,3.1.2001,Novouralsk,станция №12 -||- всё будет хорошо. -||- -||- ...
49847737,6711754,"[id68582541|Данил], я считаю, что всех чебурек...",6712471,,Grigory,Savenkov,2,18.6.1987,Luanda,None
217998036,6711754,Нужно любить своих детей. ЛЮБИТЬ.,6712676,,Galina,Yudina,1,2.12,Tolyatti,РАБОЧИЕ АВТОВАЗА -||- ПРОФСОЮЗ ЕДИНСТВО ТОЛЬЯТ...
330060237,6711556,"Дааа. "" У меня есть дырка между ног! Да, все с...",6711952,,Deniel,Semenov,2,None,Baley,None
105574664,6711556,"Как жаль, что раньше это была группа ПРО яжмат...",6712341,,Rita,Gusarova,1,8.6.1997,Saint Petersburg,"Onmyoji ASK -||- -||- Спасибо каждому, кто по..."


In [32]:
df["source"] = ["yazhemati" for i in range(len(df))]

In [33]:
df.head()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
from_id,,,,,,,,,,,
6343530,1.17367e+07,Но это был еще не конец. Я только 31 декабря 1...,11736787,"20 лет назад, 9 августа 1999 года, президент Р...",Yulia,Vershinina,1,24.9,Saint Petersburg,None,lentach
135863776,1.17367e+07,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,11736862,"20 лет назад, 9 августа 1999 года, президент Р...",Alexey,Krutikov,2,31.8,Novosibirsk,PidorasBlog -||- паблик чечни -||- Магнит гомо...,lentach
10862032,1.17367e+07,Первый срок (и часть второго даже) я одобрял е...,11736889,"20 лет назад, 9 августа 1999 года, президент Р...",Maxim,Gerasimov,2,5.8,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach
29670766,[11736731],"[id164270524|Костик], я думаю много весёлого б...",11736884,Зря боря подарил стране плешивого фашиста .,Alexey,Kovalenko,2,18.6.1998,Saint Petersburg,None,lentach
320342320,[11736787],"[id6343530|Юлия], в какой из бывших республик ...",11737428,Но это был еще не конец. Я только 31 декабря 1...,Pavel,Vladimirovich,2,26.7,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach


In [34]:
df.reset_index(inplace=True)

In [35]:
df.head()

,from_id,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
0,6343530,1.17367e+07,Но это был еще не конец. Я только 31 декабря 1...,11736787,"20 лет назад, 9 августа 1999 года, президент Р...",Yulia,Vershinina,1,24.9,Saint Petersburg,None,lentach
1,135863776,1.17367e+07,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,11736862,"20 лет назад, 9 августа 1999 года, президент Р...",Alexey,Krutikov,2,31.8,Novosibirsk,PidorasBlog -||- паблик чечни -||- Магнит гомо...,lentach
2,10862032,1.17367e+07,Первый срок (и часть второго даже) я одобрял е...,11736889,"20 лет назад, 9 августа 1999 года, президент Р...",Maxim,Gerasimov,2,5.8,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach
3,29670766,[11736731],"[id164270524|Костик], я думаю много весёлого б...",11736884,Зря боря подарил стране плешивого фашиста .,Alexey,Kovalenko,2,18.6.1998,Saint Petersburg,None,lentach
4,320342320,[11736787],"[id6343530|Юлия], в какой из бывших республик ...",11737428,Но это был еще не конец. Я только 31 декабря 1...,Pavel,Vladimirovich,2,26.7,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8741 entries, 0 to 8740
Data columns (total 12 columns):
from_id         8741 non-null int64
parent_id       8741 non-null object
comment_text    8741 non-null object
comment_id      8741 non-null int64
parent_text     8542 non-null object
first_name      8741 non-null object
last_name       8741 non-null object
sex             8741 non-null object
bdate           6875 non-null object
city            8741 non-null object
group_info      6617 non-null object
source          8741 non-null object
dtypes: int64(2), object(10)
memory usage: 478.1+ KB


## 6. Сохраняем все в файл csv

In [37]:
df.to_csv("capitalization-lentach-wall.csv", index=False)